In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
datadir="/cellar/users/snwright/Data/RareCommon/"

In [3]:
pilot = pd.read_csv(datadir+"pilot_netcoloc_results_combined.txt", sep="\t")
overlap = pd.read_csv(datadir + "pilot_overlap_results.txt", sep="\t")

FileNotFoundError: [Errno 2] File b'/cellar/users/snwright/Data/RareCommon/pilot_netcoloc_results_combined.txt' does not exist: b'/cellar/users/snwright/Data/RareCommon/pilot_netcoloc_results_combined.txt'

In [ ]:
traits = {"30630":"Apolipoprotein A", "30640":"Apolipoprotein B", "23110":"Impedance of Arm", "30830":"SHBG", "30840":"Total bilirubin", 
            "30050": "Mean corpuscular haemoglobin", "30140":"Neutrophil count", "30250": "Reticulocyte count", "30880":"Urate",
            30630:"Apolipoprotein A", 30640:"Apolipoprotein B", 23110:"Impedance of Arm", 30830:"SHBG", 30840:"Total bilirubin", 
            30050: "Mean corpuscular haemoglobin", 30140:"Neutrophil count", 30250: "Reticulocyte count", 30880:"Urate"}

pilot = pilot.replace(traits)
overlap = overlap.replace(traits)
overlap = overlap.set_index("Trait")
overlap.index.name=None

In [ ]:
overlap.describe()

In [ ]:
pilot = pilot[pilot["Trait_Common"] != "Trait_Common"]
pilot["Mean_NPS"] = pilot["Mean_NPS"].astype(float)
pilot["p_NPS"] = pilot["p_NPS"].astype(float)
pilot["p_Size"] = pilot["p_Size"].astype(float)

In [ ]:
pilot["OE"] = pilot["Size"].astype(float) / pilot["Null_Size"].astype(float)

In [ ]:
pilot.Trait_Common.unique()

In [ ]:
pilot["anno"] = pilot.p_Size.apply(lambda x: "***" if x < 0.0000001 else "**" if x < 0.00001 else "*" if x < 0.001 else "")

In [ ]:
pilot.anno.value_counts()

In [ ]:
p_size = pilot.pivot(index="Trait_Common", columns="Trait_Rare", values="p_Size")
ratio = pilot.pivot(index="Trait_Common", columns="Trait_Rare", values="OE")
anno = pilot.pivot(index="Trait_Common", columns="Trait_Rare", values="anno")


#a = a.fillna(1)

In [ ]:
order = ['Apolipoprotein A', 'Apolipoprotein B', 'SHBG', 'Total bilirubin',
       'Urate','Reticulocyte count', 'Neutrophil count','Mean corpuscular haemoglobin',
        'Impedance of Arm']
overlap.head()
p_size = p_size.loc[order, order]
ratio = ratio.loc[order, order]
anno = anno.loc[order, order]

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(6,6), gridspec_kw={"width_ratios":[3,1], "height_ratios":[1,3], "wspace":0.15, "hspace":0.15})
overlap=overlap.loc[order]
a = a.loc[order, order]
#overlap.loc[:, "Common"].plot.barh( ax=axs[1][1])
axs[0][0].bar(x=overlap.index, height=overlap["Rare"], color="grey")
axs[0][0].set_xlim(-0.5, 8.5)
axs[0][0].set_xticklabels([])
axs[0][0].set_ylabel("#Rare seeds", fontsize=12)
overlap=overlap.loc[order[::-1]]
axs[1][1].barh(y=overlap.index, width=overlap["Common"], color="grey")
axs[1][1].set_ylim(-0.5, 8.5)
axs[1][1].set_yticklabels([])
axs[1][1].set_xlabel("#Common seeds", fontsize=12)
#overlap.loc[:, "Rare"].plot.bar( ax=axs[0])
g = sns.heatmap(a, cmap='RdBu', center=1, cbar_kws={'label':"Observed/Expected", "location":"right"}, square=False, ax=axs[1][0])
g.set_facecolor("lightgrey")
axs[1][0].set_xlabel("Rare Seeds", fontsize=14)
axs[1][0].set_ylabel("Common Seeds", fontsize=14)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
axs[0][1].axis('off')
plt.savefig("/cellar/users/snwright/Data/Transfer/RVC/pilot_rare_common_bars.png", bbox_inches="tight", dpi=400)


In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(8,8), gridspec_kw={"width_ratios":[3,1], "height_ratios":[1,3], "wspace":0.05, "hspace":0.05})
overlap=overlap.loc[order]
a = a.loc[order, order]
#overlap.loc[:, "Common"].plot.barh( ax=axs[1][1])
axs[0][0].bar(x=overlap.index, height=overlap["Rare"])
axs[0][0].set_xlim(-0.5, 8.5)
axs[0][0].set_xticklabels([])
overlap=overlap.loc[order[::-1]]
axs[1][1].barh(y=overlap.index, width=overlap["Common"])
axs[1][1].set_ylim(-0.5, 8.5)
axs[1][1].set_yticklabels([])
#overlap.loc[:, "Rare"].plot.bar( ax=axs[0])
g = sns.heatmap(-1 * np.log10(a), vmax=40, cmap='viridis', cbar_kws={'label':"-log10(p-value)", "location":"left"}, square=False, ax=axs[1][0])
g.set_facecolor("lightgrey")
axs[1][0].set_xlabel("Rare Seeds", fontsize=14)
axs[1][0].set_ylabel("Common Seeds", fontsize=14)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
axs[0][1].axis('off')
#plt.savefig("/cellar/users/snwright/Data/Transfer/RVC/pilot_p_size_heatmap.png", bbox_inches="tight", dpi=400)


In [ ]:
overlap_mat = pd.read_csv(datadir+"pilot_overlap_results_matrix.txt", sep="\t")
overlap_mat = overlap_mat.replace(traits)

In [ ]:
overlap_nums = overlap_mat.pivot("TraitC", "TraitR", "Overlap")
overlap_nums = overlap_nums.loc[order, order]

In [ ]:
overlap_p = overlap_mat.pivot("TraitC", "TraitR", "Overlap_p")
overlap_p = overlap_p.loc[order, order]

In [ ]:
sns.heatmap(-1* np.log10(overlap_p), annot=overlap_nums, vmax =10, cbar_kws={'label':"-log10(p-value)"}, square=True)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ylabel("Common", fontsize=12)
plt.xlabel("Rare", fontsize=12)
plt.savefig("/cellar/users/snwright/Data/Transfer/RVC/pilot_hyperOverlap_heatmap.png", bbox_inches = "tight", dpi=400)

In [ ]:
ratio_sig = p_size < 0.01

In [ ]:
ratio_sig

In [ ]:
sns.heatmap(ratio,cmap='RdBu', cbar_kws={'label':"Observed/Expected Size"}, square=True, center=1, annot=anno.to_numpy(), fmt='s')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ylabel("Common", fontsize=12)
plt.xlabel("Rare", fontsize=12)
plt.savefig("/cellar/users/snwright/Data/Transfer/RVC/pilot_OE_heatmap.png", bbox_inches = "tight", dpi=400)

# Scope of rare variants

## Continuous traits

In [ ]:
cont = pd.read_csv(datadir+"AZdata/Table17_pan_quant.txt", sep="\t")

In [ ]:
def get_number_of_genes(data, th, phe_col, gene_col, p_col):
    data = data[data[p_col] < th]
    results = data.loc[:, (gene_col, phe_col)].drop_duplicates()
    pheno_counts = results.groupby(phe_col).count()
    gene_th = [i for i in range(1, 50)]
    traits_meeting_th = []
    for x in gene_th:
        traits_meeting_th.append( np.sum(pheno_counts >= x)[0])
    return traits_meeting_th


In [ ]:
a = get_number_of_genes(cont, 0.0001, "pheno", "geno", "p")
b = get_number_of_genes(cont, 0.0000001, "pheno", "geno", "p")

test_th = [1e-3, 1e-4,5e-5, 1e-5, 2.5e-6, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]

In [ ]:
res = pd.DataFrame({t: get_number_of_genes(cont, t, "pheno", "geno", "p") for t in test_th}, index=[i for i in range(1, 50)])
res.plot()
plt.yscale("log")
plt.vlines(5, 0, 2000, linestyle="dashed", color="black")
plt.xlabel("Minimum number of genes per trait", fontsize=12)
plt.ylabel("Number of traits meeting criteria", fontsize=12)
plt.title("Quantitative Traits")
plt.savefig("/cellar/users/snwright/Data/Transfer/RVC/quant_trait_count.png", bbox_inches="tight", dpi=400)

In [ ]:
{t: get_number_of_genes(cont, t, "pheno", "geno", "p") for t in test_th}[2.5e-6]

In [ ]:
np.sum(get_number_of_genes(cont, 0.00001, "pheno", "geno") > 50)

In [ ]:
cont.head()

# Binary traits

In [ ]:
binary = pd.read_csv(datadir+"AZdata/Table16_pan_binary.txt", sep="\t")

In [ ]:
binary.head()

In [ ]:
res = pd.DataFrame({t: get_number_of_genes(binary, t, "phenotype", "genotype", "EUR_pValue") for t in test_th}, index=[i for i in range(1, 50)])
res.plot()
plt.yscale("log")
plt.vlines(5, 0, 5000, linestyle="dashed", color="black")
plt.xlabel("Minimum number of genes per trait", fontsize=12)
plt.ylabel("Number of traits meeting criteria", fontsize=12)
plt.title("Binary Traits")
plt.savefig("/cellar/users/snwright/Data/Transfer/RVC/binary_trait_count.png", bbox_inches="tight", dpi=400)

In [ ]:
{t: get_number_of_genes(binary, t, "phenotype", "genotype", "EUR_pValue") for t in test_th}[2.5e-6]

In [ ]:
from scipy.stats import hypergeom

In [ ]:
hypergeom(100, 50, 10).sf(0)